In [1]:
import tensorflow as tf
import numpy as np
import os

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, AveragePooling1D, Conv1D, DenseFeatures, Input, Flatten, GlobalAveragePooling1D

Load all gestures from disk and label them accordingly

In [ ]:
dataset_dir = "gestures"

gestures = ["Computer", "Elephant", "Sorry", "No-Sign"]

dataset_values = []
dataset_labels = []

for gesture in gestures:
    for filename in os.listdir(f"{dataset_dir}/{gesture}"):
        with open(os.path.join(f"{dataset_dir}/{gesture}", filename), 'r') as file:
            file_data = []

            text = file.read()
            lines = text.splitlines()

            for line in lines:
                int_values = []
                for value in line.split():
                    int_values.append(int(value))

                file_data.append(int_values)

            dataset_values.append(file_data)

            if gesture == "Computer":
                dataset_labels.append([1, 0, 0, 0])
            elif gesture == "Elephant":
                dataset_labels.append([0, 1, 0, 0])
            elif gesture == "Sorry":
                dataset_labels.append([0, 0, 1, 0])
            elif gesture == "No-Sign":
                dataset_labels.append([0, 0, 0, 1])
            else:
                print("You did not change the labeling of the dataset you blithering idiot")

dataset_values = np.array(dataset_values)
dataset_labels = np.array(dataset_labels)

print(dataset_values.shape)
print(dataset_labels.shape)

Load numpy array into tensorflow dataset

In [ ]:
BATCH_SIZE = 64
SHUFFLE_BUFFER_SIZE = 2000
VALIDATION_SIZE = 300

dataset = tf.data.Dataset.from_tensor_slices((dataset_values, dataset_labels))
dataset = dataset.shuffle(SHUFFLE_BUFFER_SIZE)

train_dataset = dataset.skip(VALIDATION_SIZE).batch(BATCH_SIZE)
validation_dataset = dataset.take(VALIDATION_SIZE).batch(BATCH_SIZE)  

# for values, labels in train_dataset.take(1):
    # print(labels.numpy(), values.numpy())

In [ ]:
model = Sequential([
    Conv1D(filters = 150, strides = 3, kernel_size=70, activation='relu', input_shape=(200, 6)),
    GlobalAveragePooling1D(),
    Dense(100, activation='relu'),
    Dense(20, activation='relu'),
    Dense(4, activation='softmax')
])

model.compile(optimizer='adam', loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['accuracy'])
model.build(input_shape=(1584, 200, 6))
model.summary()
model.fit(train_dataset, validation_data=validation_dataset, epochs=25)

model.save("./model", save_format='tf')
model.save("./model.h5", save_format='h5')


In [3]:
modeldir = 'models/0.9900/model'
converter = tf.lite.TFLiteConverter.from_saved_model(modeldir)
tflite_model = converter.convert()

open("model.tflite", "wb").write(tflite_model)

Metal device set to: Apple M1 Max

systemMemory: 32.00 GB
maxCacheSize: 10.67 GB



2022-04-18 12:36:11.043436: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-04-18 12:36:11.043784: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2022-04-18 12:36:11.196444: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:357] Ignored output_format.
2022-04-18 12:36:11.196462: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:360] Ignored drop_control_dependency.
2022-04-18 12:36:11.197870: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: models/0.9900/model
2022-04-18 12:36:11.198591: I tensorflow/cc/saved_model/reader.cc:78] Reading meta graph with tags { serve }
2022-

324836